In [1]:
import pandas as pd

In [14]:
churn = pd.read_csv('customerchurn.csv')
churn.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually


In [23]:
churn.shape

(3900, 19)

In [15]:
churn.columns

Index(['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category',
       'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season',
       'Review Rating', 'Subscription Status', 'Payment Method',
       'Shipping Type', 'Discount Applied', 'Promo Code Used',
       'Previous Purchases', 'Preferred Payment Method',
       'Frequency of Purchases'],
      dtype='object')

In [24]:
churn['Category'].value_counts()

Category
Clothing       1737
Accessories    1240
Footwear        599
Outerwear       324
Name: count, dtype: int64

In [27]:
data = churn[['Frequency of Purchases', 'Age', 'Category','Gender', 'Review Rating', 'Subscription Status', 'Payment Method',  'Previous Purchases', 'Discount Applied']]
data

,Frequency of Purchases,Age,Category,Gender,Review Rating,Subscription Status,Payment Method,Previous Purchases,Discount Applied
0,Fortnightly,55,Clothing,Male,3.1,Yes,Credit Card,14,Yes
1,Fortnightly,19,Clothing,Male,3.1,Yes,Bank Transfer,2,Yes
2,Weekly,50,Clothing,Male,3.1,Yes,Cash,23,Yes
3,Weekly,21,Footwear,Male,3.5,Yes,PayPal,49,Yes
4,Annually,45,Clothing,Male,2.7,Yes,Cash,31,Yes
...,...,...,...,...,...,...,...,...,...
3895,Weekly,40,Clothing,Female,4.2,No,Cash,32,No
3896,Bi-Weekly,52,Accessories,Female,4.5,No,PayPal,41,No
3897,Quarterly,46,Accessories,Female,2.9,No,Credit Card,24,No
3898,Weekly,44,Footwear,Female,3.8,No,PayPal,24,No


In [ ]:
conda install -c conda-forge pomegranate  

  error: subprocess-exited-with-error
  
  × Building wheel for pomegranate (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [158 lines of output]
      C:\Users\STUDENT\AppData\Local\Temp\pip-build-env-keuq41yo\overlay\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\pomegranate
      copying pomegranate\callbacks.py -> build\lib.win-amd64-cpython-312\pomegranate
      copying pomegranate\io.py -> build\lib.win-amd64-cpython-312\pomegranate
      copying pomegranate\__init__.py -> build\lib.win-amd64-cpython-312\pomegranate
      creating build\lib.win-amd64-cpython-312\pomegranate\distributions
      copying pomegranate/distributions\NeuralNetworkWrapper.py -> build\lib.win-amd64-cpython-312\pomegranate/distributions
      copying pomegranate/distributions\__init__.py 


  Using cached pomegranate-0.15.0.tar.gz (5.9 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: still running...
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached Cython-3.0.12-cp312-cp312-win_amd64.whl.metadata (3.6 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached Cython-3.0.12-cp312-cp312-win_amd64.whl (2.8 MB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
Failed to build pomegranate


In [11]:
import numpy as np
import pandas as pd
from collections import defaultdict

# Preprocessing function to convert categorical data to numerical values
def preprocess_data(data):
    # Convert 'Purchased' and 'Not Purchased' states to binary (1 = Purchased, 0 = Not Purchased)
    data['Purchased'] = data['Frequency of Purchases'].apply(lambda x: 1 if x != 'Never' else 0)
    
    # Handle categorical features (e.g., Payment Method, Shipping Type) with simple encoding
    data = pd.get_dummies(data, columns=['Payment Method', 'Shipping Type', 'Category', 'Season'])
    
    # For simplicity, assume that "Previous Purchases" directly affects the likelihood of purchase
    return data

# Function to compute transition probabilities
def compute_transition_probabilities(data):
    # Create transition count matrix for states (Purchased / Not Purchased)
    transition_counts = np.zeros((2, 2))  # 2x2 matrix for state transitions

    for idx in range(1, len(data)):
        previous_purchase = data.iloc[idx-1]['Purchased']
        current_purchase = data.iloc[idx]['Purchased']
        transition_counts[previous_purchase][current_purchase] += 1

    # Normalize counts to get probabilities
    transition_probabilities = transition_counts / transition_counts.sum(axis=1, keepdims=True)
    return transition_probabilities

# Forward algorithm to compute alpha (probability of being in a particular state at time t)
def forward_algorithm(data, transition_probabilities, initial_state_prob):
    alpha = np.zeros((len(data), 2))  # Alpha matrix, where 2 is for the two states (Purchased, Not Purchased)
    
    # Initial state probabilities (probability of starting in each state)
    alpha[0] = initial_state_prob * data.iloc[0][['Purchased', 'Not Purchased']].values
    
    for t in range(1, len(data)):
        alpha[t] = np.dot(alpha[t-1], transition_probabilities)
    
    return alpha

# Backward algorithm to compute beta (backward probabilities)
def backward_algorithm(data, transition_probabilities):
    beta = np.ones((len(data), 2))  # Initialize beta to 1s (as the terminal state has uniform probability)
    
    for t in range(len(data)-2, -1, -1):  # Start from the second-to-last time step
        beta[t] = np.dot(transition_probabilities, beta[t+1])
    
    return beta

# Function to compute posterior probabilities (P(state_t | observations))
def compute_posterior(alpha, beta):
    posterior = alpha * beta
    posterior /= posterior.sum(axis=1, keepdims=True)  # Normalize to get probabilities
    return posterior

# Main function to train and test the Markov Chain model
def train_markov_chain_model(data):
    # Preprocess the data
    data = preprocess_data(data)
    
    # Compute transition probabilities
    transition_probabilities = compute_transition_probabilities(data)
    
    # Initialize initial state probabilities (e.g., equal probability of starting in each state)
    initial_state_prob = np.array([0.5, 0.5])  # Assuming equal probabilities for purchased / not purchased
    
    # Forward and backward algorithms to compute alpha and beta
    alpha = forward_algorithm(data, transition_probabilities, initial_state_prob)
    beta = backward_algorithm(data, transition_probabilities)
    
    # Compute posterior probabilities
    posterior = compute_posterior(alpha, beta)
    
    return posterior

# Example data
# data = pd.DataFrame({
#     'Customer ID': [1, 2, 3, 4, 5],
#     'Age': [55, 19, 50, 21, 45],
#     'Gender': ['Male', 'Male', 'Male', 'Male', 'Male'],
#     'Item Purchased': ['Blouse', 'Sweater', 'Jeans', 'Sandals', 'Blouse'],
#     'Category': ['Clothing', 'Clothing', 'Clothing', 'Footwear', 'Clothing'],
#     'Purchase Amount (USD)': [53, 64, 73, 90, 49],
#     'Frequency of Purchases': ['Fortnightly', 'Fortnightly', 'Weekly', 'Weekly', 'Annually'],
#     'Previous Purchases': [14, 2, 23, 49, 31],
#     'Payment Method': ['Credit Card', 'Bank Transfer', 'Cash', 'PayPal', 'Cash'],
#     'Shipping Type': ['Express', 'Express', 'Free Shipping', 'Next Day Air', 'Free Shipping'],
#     'Promo Code Used': ['Yes', 'Yes', 'Yes', 'Yes', 'Yes']
# })

# Train the Markov Chain model
posterior = train_markov_chain_model(churn)

# Print posterior probabilities for each state at each time step
print("Posterior probabilities for each state at each time step:")
print(posterior)


C:\Users\STUDENT\AppData\Local\Temp\ipykernel_22592\4121318142.py:27: RuntimeWarning: invalid value encountered in divide
  transition_probabilities = transition_counts / transition_counts.sum(axis=1, keepdims=True)


KeyError: "['Not Purchased'] not in index"

In [3]:
import pandas as pd
data = pd.read_csv("spam_ham_dataset copy.csv")
data.head(10)

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
5,2949,ham,Subject: ehronline web address change\r\nthis ...,0
6,2793,ham,Subject: spring savings certificate - take 30 ...,0
7,4185,spam,Subject: looking for medication ? we ` re the ...,1
8,2641,ham,Subject: noms / actual flow for 2 / 26\r\nwe a...,0
9,1870,ham,"Subject: nominations for oct . 21 - 23 , 2000\...",0
